In [ ]:
from PyPDF2 import PdfFileReader

reader = PdfFileReader(open(r'C:\Users\rmani\.vscode\jupyterf\PCRM\article1.pdf','rb'), strict=False) #read pdf file indicated, add an error catcher
page = reader.getPage(2) #pages are indexed
page_content = page.extractText() #extract text of the page you indicated
#print(page_content) #print the extracted text

search_term="animal testing"
if search_term in page_content:
     print("Found it!")

* using ML methods

In [23]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfFileReader
import textract
import re

#filename =r'C:\Users\rmani\.vscode\jupyterf\PCRM\article1.pdf' 

#pdfFileObj = open((filename,'rb'),strict=False)               #open allows you to read the file
pdfReader = PdfFileReader(open(r'C:\Users\rmani\.vscode\jupyterf\PCRM\article1.pdf','rb'), strict=False) #read pdf file indicated, add an error catcher
   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from

In [24]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

In [30]:
decodedtxt=text.decode("utf-8", "ignore")
#keywords = re.findall(r'[a-zA-Z]\w+',decodedtxt)
keywords = re.findall(r'[a-zA-Z]\w+',decodedtxt)
len(keywords)                               #Total keywords in document

3974

In [31]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows


In [32]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf 

In [33]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,decodedtxt)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,decodedtxt)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,decodedtxt)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,decodedtxt)[3])

In [34]:
df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf
1169,in,505,0.016543,-6.224558,-0.102971
719,an,497,0.016281,-6.208590,-0.101080
595,th,475,0.015560,-6.163315,-0.095901
932,es,441,0.014446,-6.089045,-0.087964
36,al,399,0.013070,-5.988961,-0.078278
154,the,303,0.009926,-5.713733,-0.056712
123,at,272,0.008910,-5.605802,-0.049949
553,st,272,0.008910,-5.605802,-0.049949
163,ed,256,0.008386,-5.545177,-0.046502
391,on,242,0.007927,-5.488938,-0.043513
